<p><img alt="Colaboratory logo" height="240px" src="https://upload.wikimedia.
org/wikipedia/commons/archive/f/fb/20161010213812%21Escudo-UdeA.svg" align="left" hspace="12px" vspace="0px"></p>

#**Dibujo de figuras en curvas cerradas con series de Fourier**
___________________________________________________________________
*Nombre: Santiago Montoya, Vanessa Garcia.*

*Métodos computacionales 2024-1*

*Instituto de Física.*

*Facultad de Ciencias Exactas y Naturales.*

Una figura en una curva cerrada puede considerarse como una función periódica, lo que nos permite emplear la transformada discreta de Fourier para hallar el sistema de epiciclos. Estos epiciclos no son más que círculos concatenados que crean una órbita, la cual en este caso corresponde a nuestra figura. Esto se representa como una suma de números complejos en términos de fases y amplitudes.

\begin{equation}
R_1\sin(\omega_1x)+R_1\sin(\omega_1x)+...+R_1\sin(\omega_1x)
\end{equation}

In [ ]:
#Librerias
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

\begin{equation}
X_n = \frac{1}{N} \sum_{k=0}^{N-1} x_k e^{i k \frac{2\pi}{N} n} = \frac{1}{N} \sum_{k=0}^{N-1} x_k \left[ \cos\left(k \frac{2\pi}{N} n\right) + i \sin\left(k \frac{2\pi}{N} n\right) \right]
\end{equation}




\begin{equation}
x = \frac{1}{N} \sum_{k=0}^{N-1} x_k \left[ \cos\left(k \frac{2\pi}{N} n\right) +i  \sin\left(k \frac{2\pi}{N} n\right) \right]
\end{equation}
\begin{equation}
y = \frac{1}{N} \sum_{k=0}^{N-1} y_k \left[ \cos\left(k \frac{2\pi}{N} n\right) - i \sin\left(k \frac{2\pi}{N} n\right) \right]
\end{equation}

In [ ]:
def transformada_discreta(x, y):
    N = len(x)
    X = np.zeros(N)
    Y = np.zeros(N)
    j = (N - 1) // 2  # Usar división entera

    for k in range(-j, j + 1):
        sum_x = 0
        sum_y = 0
        for n in range(N):
            arg = (2 * np.pi / N) * k * n
            sum_x += x[n] * np.cos(arg) + y[n] * np.sin(arg)
            sum_y += y[n] * np.cos(arg) - x[n] * np.sin(arg)
        X[k + j] = sum_x / N
        Y[k + j] = sum_y / N

    return X, Y

###Ahora calculamos la longitud, la fase (ángulo) y la frecuencia:

In [ ]:
def calcular_mod_pha_freq(X, Y):
    N = len(X)
    mod = np.sqrt(X**2 + Y**2)
    pha = np.arctan2(Y, X)
    freq = np.arange(-(N-1)//2, (N-1)//2 + 1)
    return mod, pha, freq

### Con las coordenadas en el plano de los complejos, la longitud(módulo), la fase (ángulo) y la frecuencia, ya podemos empezar a crear el sistema de epiciclos.
\begin{align*}
z_{\text{real}}(t) &= \sum_{j=0}^{m-1} \text{Mod}_j \cdot \cos\left(\text{Pha}_j + 2\pi \cdot \text{Freq}_j \cdot t\right) \\
z_{\text{imag}}(t) &= \sum_{j=0}^{m-1} \text{Mod}_j \cdot \sin\left(\text{Pha}_j + 2\pi \cdot \text{Freq}_j \cdot t\right) \\
\mathbf{z}(t) &= \left( z_{\text{real}}(t), z_{\text{imag}}(t) \right)
\end{align*}


In [ ]:
def calcular_epiciclos(Mod, Pha, Freq, t0, m):
    puntos = []
    for t in t0:
        z_real = 0
        z_imag = 0
        for j in range(m):
            z_real += Mod[j] * np.cos(Pha[j] + 2 * np.pi * Freq[j] * t)
            z_imag += Mod[j] * np.sin(Pha[j] + 2 * np.pi * Freq[j] * t)
        puntos.append((z_real, z_imag))
    return np.array(puntos)


Cuadrado

In [ ]:
L_1= np.array([(2, 2), (2, 1), (2, -3/5), (2, -2), (1/2, -2), (-1, -2), (-2, -2), (-2, -1/2), (-2, 1), (-2, 2), (-1, 2), (1/2, 2), (3/2, 2)])
plt.figure(figsize=(3, 3))
plt.scatter(L_1[:, 0], L_1[:, 1], color='red')
plt.grid(True)
plt.show()

In [ ]:

x = L_1[:, 0]
y = L_1[:, 1]
N = len(x)
t0 = np.arange(0, 2*np.pi, 0.0099)  # Tiempo
m = N
X, Y = transformada_discreta(x, y)
Mod, Pha, Freq = calcular_mod_pha_freq(X, Y)

# Crear la figura y el eje
fig, ax = plt.subplots(figsize=(12, 6))

ax.set_aspect('equal')
ax.set_xlim(min(x) - 1, max(x) + 1)
ax.set_ylim(min(y) - 1, max(y) + 1)
plt.title('Sistema de Epiciclos Animado con Estela')
plt.xlabel('Re(z)')
plt.ylabel('Im(z)')
plt.grid(True)

line, = ax.plot([], [], 'r-', lw=2)
point, = ax.plot([], [], 'bo')
trail, = ax.plot([], [], 'b-', lw=1)  # Línea de la estela
circles = [Circle((0, 0), 0, color='gray', fill=False, linestyle='--') for _ in range(m)]
for circle in circles:
    ax.add_patch(circle)

trail_x, trail_y = [], []

# Función de inicialización
def init():
    line.set_data([], [])
    point.set_data([], [])
    trail.set_data([], [])
    for circle in circles:
        circle.set_center((0, 0))
        circle.set_radius(0)
    return line, point, trail, *circles

# Función de actualización de la animación
def update(frame):
    L_2 = calcular_epiciclos(Mod, Pha, Freq, [t0[frame]], m)
    line.set_data(L_2[:, 0], L_2[:, 1])
    point.set_data(L_2[-1, 0], L_2[-1, 1])


    trail_x.append(L_2[-1, 0])
    trail_y.append(L_2[-1, 1])
    trail.set_data(trail_x, trail_y)

    z_real = 0
    z_imag = 0
    for j in range(m):
        z_real += Mod[j] * np.cos(Pha[j] + 2 * np.pi * Freq[j] * t0[frame])
        z_imag += Mod[j] * np.sin(Pha[j] + 2 * np.pi * Freq[j] * t0[frame])
        circles[j].set_center((z_real, z_imag))
        circles[j].set_radius(Mod[j])

    return line, point, trail, *circles


ani = FuncAnimation(fig, update, frames=len(t0), init_func=init, blit=True, interval=20)

# Mostrar la animación
HTML(ani.to_jshtml())



Corazón

In [ ]:
n_puntos=49
t=np.linspace( 0, 2*np.pi, n_puntos)
x=16*np.sin(t)**3
y=13*np.cos(t)-5*np.cos(2*t)-2*np.cos(3*t)-np.cos(4*t)
L_1=np.array(list(zip(x,y)))
plt.scatter(x,y, color='red')
plt.grid(True)
plt.show()

In [ ]:
n_puntos=49
t=np.linspace( 0, 2*np.pi, n_puntos)
x=16*np.sin(t)**3
y=13*np.cos(t)-5*np.cos(2*t)-2*np.cos(3*t)-np.cos(4*t)
N = len(x)
t0 = np.arange(0, 2*np.pi, 0.0099)  # Tiempo
m = N
X, Y = transformada_discreta(x, y)
Mod, Pha, Freq = calcular_mod_pha_freq(X, Y)


fig, ax = plt.subplots(figsize=(12, 6))

ax.set_aspect('equal')
ax.set_xlim(min(x) - 1, max(x) + 1)
ax.set_ylim(min(y) - 1, max(y) + 1)
plt.title('Sistema de Epiciclos Animado con Estela')
plt.xlabel('Re(z)')
plt.ylabel('Im(z)')
plt.grid(True)

line, = ax.plot([], [], 'r-', lw=2)
point, = ax.plot([], [], 'bo')
trail, = ax.plot([], [], 'b-', lw=1)  # Línea de la estela
circles = [Circle((0, 0), 0, color='gray', fill=False, linestyle='--') for _ in range(m)]
for circle in circles:
    ax.add_patch(circle)


trail_x, trail_y = [], []

# Función de inicialización
def init():
    line.set_data([], [])
    point.set_data([], [])
    trail.set_data([], [])
    for circle in circles:
        circle.set_center((0, 0))
        circle.set_radius(0)
    return line, point, trail, *circles

# Función de actualización de la animación
def update(frame):
    L_2 = calcular_epiciclos(Mod, Pha, Freq, [t0[frame]], m)

    line.set_data(L_2[:, 0], L_2[:, 1])
    point.set_data(L_2[-1, 0], L_2[-1, 1])


    trail_x.append(L_2[-1, 0])
    trail_y.append(L_2[-1, 1])
    trail.set_data(trail_x, trail_y)


    z_real = 0
    z_imag = 0
    for j in range(m):
        z_real += Mod[j] * np.cos(Pha[j] + 2 * np.pi * Freq[j] * t0[frame])
        z_imag += Mod[j] * np.sin(Pha[j] + 2 * np.pi * Freq[j] * t0[frame])
        circles[j].set_center((z_real, z_imag))
        circles[j].set_radius(Mod[j])

    return line, point, trail, *circles


ani = FuncAnimation(fig, update, frames=len(t0), init_func=init, blit=True, interval=20)
HTML(ani.to_jshtml())

## Bibliografía


*   Ponce, C. (2020, May 29). La órbita de Homero Simpson: Una divertida aplicación de la Transformada Discreta de Fourier. Blogspot.com; Blogger. https://bestiariotopologico.blogspot.com/2020/05/la-orbita-de-homero-simpson-una.html
*   Hsu, H. P. (2009). Análisis de Fourier. McGraw-Hill.
*   DynamicMath.xyz. (n.d.). Fourier epicycles. DynamicMath.xyz de https://www.dynamicmath.xyz/fourier-epicycles/
